In [4]:
import math
import hess
import matplotlib.pyplot as plt
import numpy as np
from hess.nets import SubnetConv

import torch
from torch import nn


In [2]:
class cifar_net(nn.Module):
    """
    Very small CNN
    """
    def __init__(self, in_dim, out_dim, k=10,
                 n_layers=5
                activation=nn.ELU()):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        module = nn.ModuleList()

        module.append(SubnetConv(in_dim, k))
        module.append(activation)
        
        for ll in range(n_layers-1):
            module.append(SubnetConv(k, k))
            
    def forward(self,x):
        return self.net(x)


NameError: name 'nn' is not defined